In [1]:
import time
import tensorflow as tf
import numpy as np
import pygame
import random
import itertools
from collections import defaultdict
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from IPython.display import clear_output

#dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
print(tf.__version__)

pygame 2.0.0.dev6 (SDL 2.0.10, python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
2.2.0


In [2]:
class snek:

    def __init__(self, size):
        self.size = size
        self.rewards = None
        self.action_space_size = 4
        self.actions = [0, 1, 2, 3]
        self.cstate = None
        self.dis = None
        self.snake = None
        self.food = None
        self.scolor = (0, 0, 255)
        self.fcolor = (255, 0, 0)
    
    def reset(self):
        self.snake = []
        state = np.zeros((self.size, self.size))
        size = self.size
        x = random.randint(0, size - 1)
        y = random.randint(0, size - 1)
        foodx = random.randint(0, size - 1)
        foody = random.randint(0, size - 1) 
        self.food = (foodx, foody)
        self.snake.append((x, y))
        self.cstate = state
        return state
    def step(self, action):
        pass

In [3]:
env = snek(64)

In [4]:
pygame.init()
dis = pygame.display.set_mode((640, 640))
pygame.display.set_caption("AI Learns to play Snake")
icon = pygame.image.load('/home/veeraraghavan/Desktop/DQL-Snake/Images/snake.png')
pygame.display.set_icon(icon)
gameover = False
while gameover != True:
    state = env.reset()
    dis.fill((0, 0, 0))
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            gameover = True
    for square in env.snake:
        pygame.draw.rect(dis, env.scolor, [square[1] * 10, square[0] * 10, 10, 10])
    pygame.draw.rect(dis, env.fcolor, [env.food[1] * 10, env.food[0] * 10, 10, 10])
    pygame.display.update()
    time.sleep(5)
pygame.quit()


In [20]:
pygame.quit()
